<a href="https://colab.research.google.com/github/Mstfayldz/MachineLearningNotebooks/blob/master/7_LogisticRegressionMultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression for Multi-class Classification

---



Logistic regresyon, genellikle iki sınıfı (binary classification) ayırmak için kullanılır. Ancak
çoklu sınıf problemlerinde (multi-class classification) de yaygın olarak uygulanabilir. Bu
durumda **One-vs-Rest** (OvR) veya **One-vs-All** stratejisi kullanılır.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df=pd.read_csv('7-cyber_attack_data.csv')

In [8]:
df.head()

,src_packet_rate,dst_packet_rate,avg_payload_size,connection_duration,tcp_flag_count,avg_interarrival_time,failed_login_attempts,unusual_port_activity_score,session_entropy,avg_response_delay,attack_type
0,-1.286132,-0.648334,1.044115,-0.469715,0.789859,-0.083727,-1.647309,-1.316412,1.011910,-0.898063,2
1,-0.222224,2.083232,1.191114,-1.354527,-0.956992,1.696028,-1.070406,0.981403,-1.628798,1.377594,0
2,-0.431963,0.375745,-1.370334,0.819214,0.345243,1.389447,-1.904130,1.292602,0.925545,0.232705,0
3,-0.912633,0.986988,-0.690042,2.014628,-0.442260,0.590347,-1.819353,1.560938,0.823755,0.517762,0
4,-0.367056,1.667892,0.879172,2.214276,1.846338,-0.894047,1.543838,0.931103,-1.015210,1.061845,1


In [9]:
df.columns

Index(['src_packet_rate', 'dst_packet_rate', 'avg_payload_size',
       'connection_duration', 'tcp_flag_count', 'avg_interarrival_time',
       'failed_login_attempts', 'unusual_port_activity_score',
       'session_entropy', 'avg_response_delay', 'attack_type'],
      dtype='object')

In [1]:
# src_packet_rate – Kaynak (gönderen) tarafından saniyede (veya belirli bir zaman diliminde) gönderilen paketlerin oranı. Yüksek değerler DDoS gibi saldırılarda görülebilir.

# dst_packet_rate – Hedef (alıcı) tarafından saniyede (veya belirli bir zaman diliminde) yanıt olarak gönderilen paketlerin oranı. Örneğin flood saldırılarında hedeften cevap çok az veya hiç gelmeyebilir.

# avg_payload_size – Gönderilen paketlerin ortalama yük (payload) boyutu (ör. bayt cinsinden). Çok küçük veya çok büyük değerler anormal trafik işareti olabilir.

# connection_duration – Bu bağlantının toplam süresi. Normal oturumlarda süre belli bir aralıkta olur, saldırı trafiğinde çok kısa (ör. port taraması) veya çok uzun olabilir.

# tcp_flag_count – TCP başlıklarında kullanılan bayrakların (SYN, ACK, FIN vb.) sayısı ya da dağılımı. Anormal TCP bayrak kombinasyonları (örn. sadece SYN’lerle dolu bağlantılar) saldırı göstergesi olabilir.

# avg_interarrival_time – İki ardışık paket arasındaki ortalama zaman farkı. Çok düşük (sürekli paket yağmuru) veya düzensiz aralıklar anomali işaretidir.

# failed_login_attempts – Oturum süresince başarısız giriş denemelerinin sayısı. Brute-force saldırılarında bu sayı genellikle yüksektir.

# unusual_port_activity_score – Port kullanımındaki anormallik skoru. Beklenmedik portlara çok sayıda bağlantı varsa bu skor yükselir (örn. port scanning).

# session_entropy – Trafiğin veri içeriği veya oturum davranışı için hesaplanmış entropi (rastgelelik) değeri. Çok yüksek entropi şifreli trafik gösterebilir; çok düşük entropi otomatik scriptler tarafından üretilmiş tekrar eden trafik göstergesi olabilir.

# avg_response_delay – İstek ile yanıt arasındaki ortalama gecikme süresi. Ağ tıkanıklığı, DoS saldırısı veya hedefin cevap verememesi gibi durumları gösterebilir.

# attack_type – Hedef değişken. Bu satırdaki trafiğin hangi saldırı türüne (veya normal) ait olduğunu etiketler.

In [10]:
df.describe()

,src_packet_rate,dst_packet_rate,avg_payload_size,connection_duration,tcp_flag_count,avg_interarrival_time,failed_login_attempts,unusual_port_activity_score,session_entropy,avg_response_delay,attack_type
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.012296,0.239737,-0.031142,0.013329,0.008900,0.016681,0.026614,0.315347,-0.045636,0.179530,1.003000
std,1.406123,1.635646,0.989677,1.001178,1.000905,1.027824,1.022217,1.364824,1.472316,0.960834,0.817104
min,-4.267039,-7.960328,-3.718638,-3.250031,-3.288725,-3.178790,-3.057529,-4.045045,-5.869039,-4.409592,0.000000
25%,-1.007421,-0.871250,-0.657668,-0.672964,-0.654218,-0.697799,-0.653787,-0.712015,-1.074499,-0.591831,0.000000
50%,0.085888,0.296278,-0.008968,0.045505,-0.032894,0.014639,0.049283,0.567003,0.121580,0.248920,1.000000
75%,1.080743,1.518676,0.615381,0.678219,0.687831,0.703139,0.713809,1.235890,1.013049,0.983239,2.000000
max,3.874738,7.168331,3.206344,2.741943,3.477044,3.600187,3.357941,4.020627,4.986178,4.112542,2.000000


In [13]:
X= df.drop("attack_type", axis=1)
y=df["attack_type"]

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y, test_size=0.3, random_state=15)


In [15]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
model.fit(X_train,y_train)
y_pred= model.predict(X_test)

In [16]:
y_pred

array([2, 0, 0, 0, 2, 0, 1, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 0, 2, 0, 0,
       0, 1, 1, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 0, 2,
       1, 1, 2, 0, 0, 0, 2, 0, 1, 2, 0, 2, 1, 2, 2, 1, 0, 0, 0, 2, 1, 2,
       1, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 1, 1, 2, 1, 0, 1, 2, 2, 0, 2, 2,
       1, 0, 1, 0, 0, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1,
       2, 1, 2, 1, 0, 1, 2, 0, 0, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2,
       2, 2, 1, 2, 0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 1, 1,
       0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0, 1, 2,
       0, 2, 2, 0, 2, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1, 2, 0, 2, 1, 2, 1, 0,
       1, 2, 0, 2, 1, 0, 1, 0, 2, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0,
       2, 0, 2, 1, 2, 1, 0, 0, 1, 1, 2, 1, 0, 2, 1, 0, 2, 1, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 2, 1, 1, 2, 0,
       2, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 1, 0, 0, 0, 2, 1, 2, 2,
       2, 1, 0, 0, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2])

In [22]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
score= accuracy_score(y_pred,y_test)
print("score:",score)
print(classification_report(y_pred,y_test))
print("confusion matrix:\n",confusion_matrix(y_pred,y_test))

score: 0.7866666666666666
              precision    recall  f1-score   support

           0       0.78      0.91      0.84        90
           1       0.74      0.73      0.73        99
           2       0.84      0.74      0.78       111

    accuracy                           0.79       300
   macro avg       0.79      0.79      0.79       300
weighted avg       0.79      0.79      0.79       300

confusion matrix:
 [[82  2  6]
 [17 72 10]
 [ 6 23 82]]


In [23]:
penalty=["l1","l2","elasticnet"]
c_values=[100,10,1,0.1,0.01] # C penalty yani cezalandırmaların lambda değeri için verilen değerdir
solver=["newton-cg","lbfgs","liblinear","newton-cholesky","sag","saga"]

In [24]:
params=dict(penalty=penalty,C=c_values,solver=solver)

In [25]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold

In [26]:
cv=StratifiedKFold()

In [27]:
grid=GridSearchCV(estimator=model,param_grid=params,cv=cv,scoring="accuracy",n_jobs=-1)

In [28]:
grid.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
250 fits failed out of a total of 450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py", line 1193, in fit
    solver = _check_solv

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [100, 10, 1, 0.1, 0.01],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [29]:
grid.best_params_

{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}

In [30]:
grid.best_score_

np.float64(0.7985714285714286)

In [31]:
y_pred=grid.predict(X_test)

In [32]:
score= accuracy_score(y_pred,y_test)
print("score:",score)
print(classification_report(y_pred,y_test))
print("confusion matrix:\n",confusion_matrix(y_pred,y_test))

score: 0.7866666666666666
              precision    recall  f1-score   support

           0       0.78      0.92      0.85        89
           1       0.71      0.75      0.73        92
           2       0.87      0.71      0.78       119

    accuracy                           0.79       300
   macro avg       0.79      0.80      0.79       300
weighted avg       0.79      0.79      0.79       300

confusion matrix:
 [[82  2  5]
 [15 69  8]
 [ 8 26 85]]


# One-vs-Rest (OvR) Stratejisi

OvR yönteminde, her sınıf için bir model oluşturulur. Her model, o sınıfı diğer tüm
sınıflardan ayırt etmeye çalışır. Eğer elimizde K tane sınıf varsa, K adet ikili (binary)
sınıflayıcı eğitilir.


# Model Eğitimi Süreci
Her model şu şekilde eğitilir:

* Model M1: Class 1 vs. Rest

*  Model M2: Class 2 vs. Rest
*  Model M3: Class 3 vs. Rest

Her model kendi sınıfına ait örnekleri pozitif (1) olarak, diğer tüm sınıflara ait örnekleri
negatif (0) olarak etiketler.
Elde edilen tüm modeller yeni bir veri üzerinde test edilir. Hangi model en yüksek
olasılığı üretirse, veri o sınıfa atanır


# Tahmin Aşaması
Yeni bir test verisi, her modele ayrı ayrı verilir ve her model ilgili sınıfın olasılığını hesaplar:

* Model M1: P1 = 0.25
* Model M2: P2 = 0.20
* Model M3: P3 = 0.55

En yüksek olasılığa sahip model (M3) seçilir ve test verisi bu modele ait sınıfa (Class
3) atanır

# One-vs-One (OvO)

OvOyaklaşımında, her iki sınıf çifti için ayrı bir model eğitilir. Yani K sınıf için toplamda:

K(K −1)/2

adet model oluşturulur. Her model sadece iki sınıf arasında ayırım yapar. Test verisi
her modelden bir "oy" alır; hangi sınıf daha fazla oy alırsa o sınıf atanır.
Örnek: Eğer sınıflar A, B, ve C ise:

* Model 1: A vs B
* Model 2: A vs C
* Model 3: B vs C

OvO yöntemi, veri setindeki sınıflar arasında daha dengeli modeller oluşturulmasına
olanak sağlar. Ancak model sayısı hızla artabileceği için büyük sınıf sayısına sahip veri
setlerinde maliyetli olabilir.


In [33]:
from sklearn.multiclass import OneVsOneClassifier , OneVsRestClassifier


In [34]:
oneVSone= OneVsOneClassifier(LogisticRegression())# içine modeli tanımlamamız gerekiyor

In [35]:
oneVSrest= OneVsRestClassifier(LogisticRegression())

In [36]:
oneVSone.fit(X_train,y_train)
y_pred= oneVSone.predict(X_test)

In [37]:
score= accuracy_score(y_pred,y_test)
print("score:",score)
print(classification_report(y_pred,y_test))
print("confusion matrix:\n",confusion_matrix(y_pred,y_test))

score: 0.7966666666666666
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        91
           1       0.74      0.76      0.75        95
           2       0.85      0.73      0.78       114

    accuracy                           0.80       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.80      0.80      0.80       300

confusion matrix:
 [[84  3  4]
 [12 72 11]
 [ 9 22 83]]


In [38]:
oneVSrest.fit(X_train,y_train)
y_pred= oneVSrest.predict(X_test)

In [39]:
score= accuracy_score(y_pred,y_test)
print("score:",score)
print(classification_report(y_pred,y_test))
print("confusion matrix:\n",confusion_matrix(y_pred,y_test))

score: 0.7833333333333333
              precision    recall  f1-score   support

           0       0.79      0.91      0.85        91
           1       0.71      0.73      0.72        94
           2       0.85      0.72      0.78       115

    accuracy                           0.78       300
   macro avg       0.78      0.79      0.78       300
weighted avg       0.79      0.78      0.78       300

confusion matrix:
 [[83  2  6]
 [16 69  9]
 [ 6 26 83]]
